In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../..')

from utils.categorize_data import (setup_dti_cat, categorize_cltv, categorize_property_value_ratio, 
                                   calculate_prop_zscore, categorize_age, categorize_sex, 
                                   categorize_underwriter, categorize_loan_term, categorize_lmi)

### 1. Import Cleaned Data
- Rows: 17,545,457
- Columns: 87

In [3]:
hmda19_df = pd.read_csv('../../data/hmda_lar/cleaned_data/1_hmda2019_210823.csv', dtype = str)

hmda19_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17545457 entries, 0 to 17545456
Data columns (total 87 columns):
 #   Column                                    Dtype 
---  ------                                    ----- 
 0   activity_year                             object
 1   lei                                       object
 2   derived_msa_md                            object
 3   state_code                                object
 4   county_code                               object
 5   census_tract                              object
 6   conforming_loan_limit                     object
 7   action_taken                              object
 8   purchaser_type                            object
 9   preapproval                               object
 10  loan_type                                 object
 11  loan_purpose                              object
 12  lien_status                               object
 13  reverse_mortgage                          object
 14  open_end_line_of

### 2. Join with Lender Info

In [4]:
lender_def = pd.read_csv('../../data/supplemental_hmda_data/cleaned/lender_definitions_em210513.csv', 
                         dtype = str)

lender_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5508 entries, 0 to 5507
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   lei              5508 non-null   object
 1   agency_code      5508 non-null   object
 2   respondent_name  5508 non-null   object
 3   lar_count        5508 non-null   object
 4   assets           5508 non-null   object
 5   lender_def       5508 non-null   object
 6   lender_size      5508 non-null   object
 7   con_apps         5148 non-null   object
dtypes: object(8)
memory usage: 344.4+ KB


In [5]:
lender_def2 = lender_def[['lei', 'lar_count', 'assets', 'lender_def', 'con_apps']].copy()
lender_def2.head(1)

,lei,lar_count,assets,lender_def,con_apps
0,254900ZBZ4M7TCGJWL09,2,48362,2,1.0


In [6]:
hmda19_df = pd.merge(hmda19_df, lender_def2, how = 'left', on = ['lei'])

Every record in HMDA data has a lender match. There are no missing values after the join. 

In [7]:
hmda19_df['lar_count'].isnull().values.sum()

0

#### Lender Definition
Only 30,000 records, less than one percent,  in overall HMDA data come from no definitions for lenders.
- 1: Banks
- 2: Credit Union
- 3: Independent Mortgage Companies
- 4: No definition 

In [8]:
print(hmda19_df['lender_def'].value_counts(dropna = False, normalize = True) * 100)

3    49.872243
1    39.705857
2    10.256877
4     0.165023
Name: lender_def, dtype: float64


### 3. Adding Metro Definitions

In [9]:
counties_df = pd.read_csv('../../data/census_data/county_to_metro_crosswalk/clean/all_counties_210804.csv',
                          dtype = str)

counties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225 entries, 0 to 3224
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   fips_state_code   3224 non-null   object
 1   fips_county_code  3224 non-null   object
 2   county_name       3220 non-null   object
 3   state_name        3220 non-null   object
 4   cbsa_code         1916 non-null   object
 5   cbsa_title        1916 non-null   object
 6   csa_code          1256 non-null   object
 7   csa_title         1256 non-null   object
 8   metro_type        3225 non-null   object
 9   metro_code        1916 non-null   object
 10  metro_name        1916 non-null   object
 11  metro_pop         1915 non-null   object
 12  metro_percentile  3225 non-null   object
 13  metro_type_def    3225 non-null   object
dtypes: object(14)
memory usage: 352.9+ KB


In [10]:
counties_df2 = counties_df[['fips_state_code', 'fips_county_code', 'metro_code', 'metro_type_def',
                            'metro_percentile']].copy()

counties_df2 = counties_df2.rename(columns = {'fips_state_code': 'state_fips', 
                                              'fips_county_code': 'county_fips'})

counties_df2.head(1)

,state_fips,county_fips,metro_code,metro_type_def,metro_percentile
0,02,013,NaN,4,000


#### Metro Percentile Definitions
Majority of applications come from metros in the 80th percentile or larger ones.

- 111: Micro
- 000: No Metro
- 99: 99th percentile
- 9: 90th percentile

In [11]:
hmda19_df = pd.merge(hmda19_df, counties_df2, how = 'left', on = ['state_fips', 'county_fips'])

hmda19_df['metro_percentile'].value_counts(dropna = False, normalize = True) * 100

9      35.545794
8      15.468893
99     10.075007
7       9.244507
111     6.117749
000     5.604682
6       5.547379
5       3.773051
4       2.600383
3       2.148174
2       1.668335
1       1.258109
0       0.947938
Name: metro_percentile, dtype: float64

### 4. Add Property Value by County

In [12]:
prop_values_df = pd.read_csv('../../data/census_data/property_values/' + 
                             'ACSDT5Y2019.B25077_data_with_overlays_2021-06-23T115616.csv', dtype = str)

prop_values_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GEO_ID       3221 non-null   object
 1   NAME         3221 non-null   object
 2   B25077_001E  3221 non-null   object
 3   B25077_001M  3221 non-null   object
dtypes: object(4)
memory usage: 100.8+ KB


#### First pass at cleaning median property value data

In [13]:
prop_values_df2 = prop_values_df[(prop_values_df['GEO_ID'] != 'id')]

prop_values_df3 = prop_values_df2.rename(columns = {'B25077_001E': 'median_value', 
                                                    'B25077_001M': 'median_value_moe'})

prop_values_df3['state_fips'] = prop_values_df3['GEO_ID'].str[9:11]
prop_values_df3['county_fips'] = prop_values_df3['GEO_ID'].str[11:]

prop_values_df4 = prop_values_df3[['state_fips', 'county_fips', 'median_value']].copy()


prop_values_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220 entries, 1 to 3220
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state_fips    3220 non-null   object
 1   county_fips   3220 non-null   object
 2   median_value  3220 non-null   object
dtypes: object(3)
memory usage: 100.6+ KB


#### Convert property value to numeric
- No property value for these two counties

In [14]:
prop_values_df4[(prop_values_df4['median_value'] == '-')]

,state_fips,county_fips,median_value
549,15,005,-
2674,48,301,-


In [15]:
prop_values_df4.loc[(prop_values_df4['median_value'] != '-'), 'median_prop_value'] = prop_values_df4['median_value']
prop_values_df4.loc[(prop_values_df4['median_value'] == '-'), 'median_prop_value'] = np.nan
prop_values_df4['median_prop_value'] = pd.to_numeric(prop_values_df4['median_prop_value'])

prop_values_df4[(prop_values_df4['median_prop_value'].isnull())]

,state_fips,county_fips,median_value,median_prop_value
549,15,005,-,NaN
2674,48,301,-,NaN


In [16]:
hmda19_df = pd.merge(hmda19_df, prop_values_df4, how = 'left', on = ['state_fips', 'county_fips'])

In [17]:
hmda19_df.loc[(hmda19_df['property_value'] != 'Exempt'), 'prop_value'] = hmda19_df['property_value']

hmda19_df.loc[(hmda19_df['property_value'] == 'Exempt'), 'prop_value'] = np.nan

hmda19_df['prop_value'] = pd.to_numeric(hmda19_df['prop_value'])

### 5. Add Race and Ethnicity Demographic per Census Tract

In [18]:
race_df = pd.read_csv('../../data/census_data/racial_ethnic_demographics/clean/tract_race_pct2019_210204.csv',
                     dtype = str)

race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year                   74001 non-null  object
 1   state                  74001 non-null  object
 2   county                 74001 non-null  object
 3   tract                  74001 non-null  object
 4   total_estimate         74001 non-null  object
 5   total_moe              74001 non-null  object
 6   white_alone            74001 non-null  object
 7   white_alone_moe        74001 non-null  object
 8   black_alone            74001 non-null  object
 9   black_alone_moe        74001 non-null  object
 10  native_alone           74001 non-null  object
 11  native_alone_moe       74001 non-null  object
 12  asian_alone            74001 non-null  object
 13  asian_moe              74001 non-null  object
 14  pacislander_alone      74001 non-null  object
 15  pacislander_alone_m

In [19]:
race_df['white_pct'] = pd.to_numeric(race_df['white_pct'])

race_df['census_tract'] = race_df['state'] + race_df['county'] + race_df['tract']

race_df2 = race_df[['census_tract', 'total_estimate', 'white_pct', 'black_pct', 'native_pct', 'latino_pct', 
                    'asian_pct', 'pacislander_pct', 'othercb_pct', 'asiancb_pct']].copy()

race_df2.sample(2, random_state = 303)

,census_tract,total_estimate,white_pct,black_pct,native_pct,latino_pct,asian_pct,pacislander_pct,othercb_pct,asiancb_pct
6006,48479000300,1859,0.376547,0.21516944593867668,0.0,99.40828402366864,0.0,0.0,0.0,0.0
23211,39057240100,3026,90.647720,4.758757435558493,0.0,2.4124256444150696,0.0,0.0,2.181097157964309,0.0


#### Create White Gradiant

In [20]:
race_df2.loc[(race_df2['white_pct'] > 75), 'diverse_def'] = '1'

race_df2.loc[(race_df2['white_pct'] <= 75) & (race_df2['white_pct'] > 50), 'diverse_def'] = '2'

race_df2.loc[(race_df2['white_pct'] <= 50) & (race_df2['white_pct'] > 25), 'diverse_def'] = '3'

race_df2.loc[(race_df2['white_pct'] <= 25), 'diverse_def'] = '4'

race_df2.loc[(race_df2['white_pct'].isnull()), 'diverse_def'] = '5'

race_df2['diverse_def'].value_counts(dropna = False)

1    31608
2    17416
4    13573
3    10701
5      703
Name: diverse_def, dtype: int64

- 0: No census data there
- NaN: Records that don't find a match in the census data

In [21]:
hmda19_df = pd.merge(hmda19_df, race_df2, how = 'left', on = ['census_tract'])

Convert the NaN to 0's

In [22]:
hmda19_df.loc[(hmda19_df['diverse_def'].isnull()), 'diverse_def'] = '0'

hmda19_df['diverse_def'].value_counts(dropna = False)

1    8245326
2    4695974
3    2419878
4    1797303
0     386603
5        373
Name: diverse_def, dtype: int64

### 7. Clean Debt-to-Income Ratio

In [23]:
dti_df = pd.DataFrame(hmda19_df['debt_to_income_ratio'].value_counts(dropna = False)).reset_index().\
         rename(columns = {'index': 'debt_to_income_ratio', 'debt_to_income_ratio': 'count'})

### Convert the nulls for cleaning purposes
dti_df = dti_df.fillna('null')

dti_df.head(2)

,debt_to_income_ratio,count
0,null,5958774
1,20%-<30%,2020425


In [24]:
### Running function to organize debt-to-income ratio
dti_df['dti_cat'] = dti_df.apply(setup_dti_cat, axis = 1)

dti_df.head(2)

,debt_to_income_ratio,count,dti_cat
0,null,5958774,6
1,20%-<30%,2020425,1


In [25]:
### Drop count column and replace the null values back to NaN
dti_df2 = dti_df.drop(columns = ['count'], axis = 1)
dti_df2 = dti_df2.replace('null', np.nan)

dti_df2.head(2)

,debt_to_income_ratio,dti_cat
0,NaN,6
1,20%-<30%,1


A third of entire dataset is null, when it comes to DTI ratio.

In [26]:
hmda19_df = pd.merge(hmda19_df, dti_df2, how = 'left', on = ['debt_to_income_ratio'])

hmda19_df['dti_cat'].value_counts(dropna = False, normalize = True) * 100

6    33.961920
1    26.766444
2    15.020167
3    12.738312
4     8.863337
5     2.649820
Name: dti_cat, dtype: float64

### 8. Combine Loan-to-Value Ratio

In [27]:
cltv_df = pd.DataFrame(hmda19_df['combined_loan_to_value_ratio'].value_counts(dropna = False)).reset_index().\
          rename(columns = {'index': 'combined_loan_to_value_ratio', 'combined_loan_to_value_ratio': 'count'})

### Convert cltv to numeric
cltv_df.loc[(cltv_df['combined_loan_to_value_ratio'] != 'Exempt'), 'cltv_ratio'] =\
            cltv_df['combined_loan_to_value_ratio']

cltv_df['cltv_ratio'] = pd.to_numeric(cltv_df['cltv_ratio'])

#### Downpayment Flag
- 1: 20 percent or more downpayment
- 2: Less than 20 percent
- 3: Nulls

In [28]:
cltv_df['downpayment_flag'] = cltv_df.apply(categorize_cltv, axis = 1)
cltv_df2 = cltv_df.drop(columns = ['count', 'cltv_ratio'], axis = 1)


hmda19_df = pd.merge(hmda19_df, cltv_df2, how = 'left', on = ['combined_loan_to_value_ratio'])
hmda19_df['downpayment_flag'].value_counts(dropna = False)

3    6434599
1    6325426
2    4785432
Name: downpayment_flag, dtype: int64

### 9. Property Value Ratio Z-Score

Property value ratios are more normally distributed than raw property values. Because there's they are normally distributed below the 10th ratio, I will use the z-scores and place them into buckets based on those z-scores.

In [29]:
property_value_df = pd.DataFrame(hmda19_df.groupby(by = ['state_fips', 'county_fips', 'property_value',
                    'prop_value', 'median_prop_value'], dropna = False).size()).reset_index().\
                     rename(columns = {0: 'count'})

In [30]:
property_value_df['property_value_ratio'] = property_value_df['prop_value'].\
                                            div(property_value_df['median_prop_value']).round(3)

property_value_df['prop_zscore'] = property_value_df.apply(calculate_prop_zscore, axis = 1).round(3)

property_value_df['prop_value_cat'] = property_value_df.apply(categorize_property_value_ratio, axis = 1)

property_value_df.sample(3, random_state = 303)

,state_fips,county_fips,property_value,prop_value,median_prop_value,count,property_value_ratio,prop_zscore,prop_value_cat
251094,55,067,245000,245000.0,113900.0,13,2.151,0.745,3
205653,47,161,225000,225000.0,123100.0,18,1.828,0.389,3
32576,08,083,555000,555000.0,222800.0,5,2.491,1.120,4


In [31]:
property_value_df2 = property_value_df[['state_fips', 'county_fips', 'property_value',
                                        'median_prop_value', 'property_value_ratio', 'prop_zscore',
                                        'prop_value_cat']].copy()

In [32]:
hmda19_df = pd.merge(hmda19_df, property_value_df2, how = 'left', on = ['state_fips', 'county_fips',
                     'property_value', 'median_prop_value'])

### 10. Applicant Age

- [9999](https://s3.amazonaws.com/cfpb-hmda-public/prod/help/2018-public-LAR-code-sheet.pdf): No Co-applicant
- 8888: Not Applicable

In [33]:
age_df = pd.DataFrame(hmda19_df['applicant_age'].value_counts(dropna = False)).reset_index().\
         rename(columns = {'index': 'applicant_age', 'applicant_age': 'count'})

In [34]:
age_df['applicant_age_cat'] = age_df.apply(categorize_age, axis = 1)

age_df = age_df.drop(columns = ['count'], axis = 1)

#### Age Categories
- 1: Less than 25
- 2: 25 through 34
- 3: 35 through 44
- 4: 45 through 54
- 5: 55 through 64
- 6: 65 through 74
- 7: Greater than 74
- 8: Not Applicable

In [35]:
hmda19_df = pd.merge(hmda19_df, age_df, how = 'left', on = ['applicant_age'])

hmda19_df['applicant_age_cat'].value_counts(dropna = False)

3    3862374
4    3525316
2    3117932
5    2819358
6    1648210
8    1530071
7     590659
1     451537
Name: applicant_age_cat, dtype: int64

### 11. Income and Loan Amount Log

In [36]:
hmda19_df['income'] = pd.to_numeric(hmda19_df['income'])
hmda19_df['loan_amount'] = pd.to_numeric(hmda19_df['loan_amount'])

hmda19_df['income_log'] = np.log(hmda19_df['income'])
hmda19_df['loan_log'] = np.log(hmda19_df['loan_amount'])

/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/emmanuel/.pyenv/versions/3.8.0/envs/redlining/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### 12. Applicant Sex
- 1: Male
- 2: Female
- 3: Information not provided
- 4: Not Applicable
- 5: No Co-Applicable
- 6: Marked Both

In [37]:
sex_df = pd.DataFrame(hmda19_df['applicant_sex'].value_counts(dropna = False)).reset_index().\
         rename(columns = {'index': 'applicant_sex', 'applicant_sex': 'count'})

In [38]:
sex_df = sex_df.drop(columns = ['count'], axis = 1)

sex_df['applicant_sex_cat'] = sex_df.apply(categorize_sex, axis = 1)

#### New applicant sex categories
- 1: Male
- 2: Female
- 3: Not applicable
- 4: Makred both sexes

In [39]:
hmda19_df = pd.merge(hmda19_df, sex_df, how = 'left', on = ['applicant_sex'])

hmda19_df['applicant_sex_cat'].value_counts(dropna = False)

1    9831099
2    4847879
3    2857746
6       8733
Name: applicant_sex_cat, dtype: int64

### 13. Automated Underwiting systems
- 1: Only one AUS was used
- 2: Same AUS was multiple times
- 3: Different AUS were used
- 4: Exempt

In [40]:
hmda19_df['aus_cat'].value_counts(dropna = False)

1    16144858
3      778268
4      471452
2      150879
Name: aus_cat, dtype: int64

In [41]:
underwriter_df = pd.DataFrame(hmda19_df.groupby(by = ['aus_1', 'aus_cat']).size()).reset_index().\
                 rename(columns = {0: 'count'})

underwriter_df['main_aus'] = underwriter_df.apply(categorize_underwriter, axis = 1)

underwriter_df = underwriter_df.drop(columns = ['count'], axis = 1)

#### Main Aus
- 1: Desktop Underwriter
- 2: Loan Prospector
- 3: Technology Open to Approved Lenders
- 4: Guaranteed Underwriting System
- 5: Other
- 6: No main Aus
- 7: Not Applicable

In [42]:
hmda19_df = pd.merge(hmda19_df, underwriter_df, how = 'left', on = ['aus_1', 'aus_cat'])

hmda19_df['main_aus'].value_counts(dropna = False)

7    8037522
1    5695977
2    1347808
3     813788
6     778268
5     774798
4      97296
Name: main_aus, dtype: int64

### 14. Loan Term

In [43]:
loanterm_df = pd.DataFrame(hmda19_df['loan_term'].value_counts(dropna = False)).reset_index().\
              rename(columns = {'index': 'loan_term', 'loan_term': 'count'})

loanterm_df.loc[(loanterm_df['loan_term'] != 'Exempt'), 'em_loan_term'] = loanterm_df['loan_term']

loanterm_df['em_loan_term'] = pd.to_numeric(loanterm_df['em_loan_term'])

In [44]:
loanterm_df['mortgage_term'] = loanterm_df.apply(categorize_loan_term, axis = 1)

loanterm_df = loanterm_df.drop(columns = ['count', 'em_loan_term'])

#### Mortgage Term
- 1: 30 year mortgage
- 2: Less than 30 years
- 3: More than 30 years
- 4: Not applicable

In [45]:
hmda19_df = pd.merge(hmda19_df, loanterm_df, how = 'left', on = ['loan_term'])

hmda19_df['mortgage_term'].value_counts(dropna = False)

1    12809017
2     3650072
4      704904
3      381464
Name: mortgage_term, dtype: int64

### 15. Tract MSA Income Percentage

In [46]:
tractmsa_income_df = pd.DataFrame(hmda19_df['tract_to_msa_income_percentage'].value_counts(dropna = False)).\
                     reset_index().rename(columns = {'index': 'tract_to_msa_income_percentage', 
                                                     'tract_to_msa_income_percentage': 'count'})

tractmsa_income_df['tract_msa_ratio'] = pd.to_numeric(tractmsa_income_df['tract_to_msa_income_percentage'])

In [47]:
tractmsa_income_df['lmi_def'] = tractmsa_income_df.apply(categorize_lmi, axis = 1)

tractmsa_income_df = tractmsa_income_df.drop(columns = ['count', 'tract_msa_ratio'], axis = 1)

#### LMI Definition
- 1: Low
- 2: Moderate
- 3: Middle
- 4: Upper
- 5: None

In [48]:
hmda19_df = pd.merge(hmda19_df, tractmsa_income_df, how = 'left', on = ['tract_to_msa_income_percentage'])

hmda19_df['lmi_def'].value_counts(dropna = False)

3    7548572
4    6610399
2    2553233
1     429088
5     404165
Name: lmi_def, dtype: int64

### 16. Filter: 

#### For Conventional and FHA loans that first-lien, one-to-four unit, site built unites for home purchase where the applicant is going to live in that property

In [49]:
one_to_four = ['1', '2', '3', '4']

hmda19_df2 = hmda19_df[((hmda19_df['loan_type'] == '1') | (hmda19_df['loan_type'] == '2'))\
                      & (hmda19_df['occupancy_type'] == '1') &\
                        (hmda19_df['total_units'].isin(one_to_four)) &\
                        (hmda19_df['loan_purpose'] == '1') &\
                        (hmda19_df['action_taken'] != '6') &\
                        (hmda19_df['construction_method'] == '1') &\
                        (hmda19_df['lien_status'] == '1') &\
                        (hmda19_df['business_or_commercial_purpose'] != '1')].copy()

print('hmda19_df: ' + str(len(hmda19_df)))
print('hmda19_df2: ' + str(len(hmda19_df2)))

hmda19_df: 17545457
hmda19_df2: 4529912


### 17. Write new dataframe to CSV

In [50]:
hmda19_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4529912 entries, 222 to 17545397
Columns: 119 entries, activity_year to lmi_def
dtypes: float64(8), int64(1), object(110)
memory usage: 4.1+ GB


In [51]:
hmda19_df2.to_csv('../../data/hmda_lar/cleaned_data/2_hmda2019_210823.csv', index = False)